## BIG QUESTIONS
best syntax?
how do deal with adding new parameters, should they be part of the sequence? would you ever add a parameter mid sequency?
there is parameter object and parameter analysisStep?

In [1]:
import functioneer as fn
from scipy.optimize import LinearConstraint

In [2]:
### Choosing a Function to Analyze
# Functioneer is designed to analyze ANY function(s) with ANY number of inputs and outputs. For the following examples, the [Rosenbrock Function](https://en.wikipedia.org/wiki/Rosenbrock_function) is used for its relative simplicity, 4 inputs (plenty to play with) and its historical significance as an optimization benchmark.

# Rosenbrock function (known minimum of 0 at: x=1, y=1, a=1, b=100)
def rosenbrock(x, y, a, b):
    return (a-x)**2 + b*(y-x**2)**2

In [3]:
### Example 1: The Basics (Defining Parameters and Executing a Function)
# Set up an *analysis sequence* by defining four parameters (the inputs needed for the Rosenbrock function), then executing the function (with parameter ids matched to kwargs)

# Create new analysis
anal = fn.AnalysisModule() # its not ānal is anál!

# Define analysis sequence
anal.add.define('a', 1) # Define parameter 'a'
anal.add.define('b', 100) # Define parameter 'b'
anal.add.define('x', 1) # Define parameter 'x'
anal.add.define('y', 1) # Define parameter 'y'

anal.add.execute(func=rosenbrock, assign_to='rosen') # Execute function with parameter ids matched to kwargs

# Run the analysis sequence
results = anal.run()

print(results['df'])

done with analysis!
   runtime  a    b  x  y  rosen                   datetime
0      0.0  1  100  1  1      0 2025-06-23 16:13:20.764028


In [4]:
### Example 2: Single Parameter Forks (Testing Variations of a Parameter)
# If you want to test a set of values for a parameter you can create a *fork* in the *analysis sequence*. This splits the analysis into multiple *branches*, each exploring different values for a the given parameter.

# Say we want to evaluate and plot the Rosenbrock surface over the x-y domain. Let's evaluate Rosenbrock a grid where x=(0, 1, 2) and y=(1, 10) which should result in 6 final *branches* / *leaves*...

# Note: some boiler plate can be removed by defining initial parameters in the AnalysisModule declaration

# Create new analysis
init_params = dict(a=1, b=100, x=1, y=1) # initial parameters will be overwritten by forks, optimizations, etc
anal = fn.AnalysisModule(init_params)

# Define analysis sequence
anal.add.fork('x', value_set=(0, 1, 2)) # Fork analysis, create a branch for each value of 'x': 0, 1, 2
anal.add.fork('y', value_set=(1, 10)) # Fork analysis, create a branch for each value of 'y': 1, 10

anal.add.execute(func=rosenbrock, output_param_ids='rosen') # Execute function (for each branch) with parameters matched to kwargs

# Run the analysis sequence
results = anal.run()

print(results['df'].drop(columns='datetime'))

TypeError: AnalysisModule.AddNamespace.ExecuteNamespace.__call__() got an unexpected keyword argument 'output_param_ids'

In [4]:
### Example 3: Optimization
# Let's say you want to find the local minimum of the Rosenbrock (optimize `x` and `y`) for several variations of `a` and `b` (different flavors Rosenbrock functions). You would fork the analysis at parameters `a` and `b`, then perform an optimization on each branch.

# Create new analysis
anal = fn.AnalysisModule(dict(x=0, y=0))

# Define analysis sequence
anal.add.fork('a', value_set=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add.fork('b', value_set=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200

anal.add.optimize(func=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y'))

# Run the analysis sequence
results = anal.run()
print(results['df'].drop(columns='datetime'))

RuntimeError: Analysis failed: Error in Fork step at index 0: Fork step at index 0 returned invalid output: Expected tuple of ParameterSet, got <class 'list'>

In [ ]:
### Example 4: Multi-parameter Forks
# If you want to test specific combinations of parameters (instead of creating a grid) use a *multi-parameter fork*. The following will result in 3 branches: (a=0, b=0), (a=1, b=100), (a=2, b=200)

# Create new analysis
anal = fn.AnalysisModule(dict(a=1, b=100))

# Define analysis sequence
anal.add.fork.multi(('x', 'y'), value_sets=((0, 1, 2), (0, 10, 20))) # Fork analysis, create a branch for each value of 'y': 1, 10

anal.add.execute(func=rosenbrock, output_param_ids='rosen') # Execute function (for each branch) with parameters matched to kwargs

# Run the analysis sequence
results = anal.run()
print(results['df'].drop(columns='datetime'))

done with analysis!
   runtime  a    b  x   y  rosen
0      0.0  1  100  0   0      1
1      0.0  1  100  1  10   8100
2      0.0  1  100  2  20  25601


In [ ]:
### Example 5: Analysis Steps can be Conditional
# Any *analysis step* can be given a conditional function that must return true at runtime or else the *analysis step* will be skipped. An example use case is when you want to skip an expensive *analysis step* if the parameters aren't looking "good".
# As an arbitrary example, assume that we only care about cases where the optimized value of `y` is above 0.5. Also assume `expensive_func` is costly to run and we want to avoid running it when `y<0.5`. 

# Create new analysis
anal = fn.AnalysisModule(dict(x=0, y=0))

# Define analysis sequence
anal.add.fork('a', value_set=(1, 2))
anal.add.fork('b', value_set=(0, 100, 200))
anal.add.optimize(func=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y'))

# Only evaluate 'expensive_func' if the optimized 'y' is above 0.5
expensive_func = lambda x, y: x+y
anal.add.execute(func=expensive_func, output_param_ids='expensive_param', condition=lambda y: y>0.5)

results = anal.run()
print(results['df'].drop(columns='datetime'))

done with analysis!
    runtime         x         y  a    b         rosen  expensive_param
0  0.014712  1.000000  0.000000  1    0  4.930381e-32              NaN
1  0.011060  0.999763  0.999523  1  100  5.772481e-08         1.999286
2  0.007107  0.999939  0.999873  1  200  8.146869e-09         1.999811
3  0.001328  2.000000  0.000000  2    0  0.000000e+00              NaN
4  0.008017  1.999731  3.998866  2  100  4.067518e-07         5.998596
5  0.002747  1.999554  3.998225  2  200  2.136755e-07         5.997779


In [ ]:
### Example 3: Optimization
# Let's say you want to find the local minimum of the Rosenbrock (optimize `x` and `y`) for several variations of `a` and `b` (different flavors Rosenbrock functions). You would fork the analysis at parameters `a` and `b`, then perform an optimization on each branch.

# Create new analysis
anal = fn.AnalysisModule(dict(x=0, y=0))

# Define analysis sequence
anal.add.fork('a', value_set=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add.fork('b', value_set=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200

# basic default assign_to
anal.add.optimize(func=rosenbrock, opt_param_ids=('x', 'y'))

# direction
neg_ros = lambda x, y, a, b: -rosenbrock(x, y, a, b)
anal.add.optimize(func=neg_ros, assign_to='neg_ros', opt_param_ids=('x', 'y'), direction='max')

# other opt methods
linear_constraint = LinearConstraint([1, 1], 1, 1)
anal.add.optimize(func=rosenbrock, assign_to='rosen_NM', opt_param_ids=('x', 'y'), optimizer='Nelder-Mead')


# options
options = dict()
anal.add.define('a', 57)
anal.add.optimize(func=rosenbrock, assign_to='rosen_low_tol', opt_param_ids=('x', 'y'), tol=1e-2)


# 

# Run the analysis sequence
results = anal.run()
print(results['df'].drop(columns='datetime'))

RuntimeError: Optimization error: Optimization did not converge: Iteration limit reached

In [ ]:
### Figuring out execute and optimization options API

# Create new analysis
init_params = dict(a=1, b=100, x=1, y=1) # initial parameters will be overwritten by forks, optimizations, etc
anal = fn.AnalysisModule(init_params)

# Define analysis sequence
anal.add.fork('x', value_set=(0, 1, 2)) # Fork analysis, create a branch for each value of 'x': 0, 1, 2
anal.add.fork('y', value_set=(1, 10)) # Fork analysis, create a branch for each value of 'y': 1, 10

anal.add.execute(
    func=rosenbrock, 
    assign_output_to='rosen' # alt arg names: output_param output_param_id
) # Execute function (for each branch) with parameters matched to kwargs

anal.add.execute(func=rosenbrock, func_return_type='dict') # Execute function (for each branch) with parameters matched to kwargs
anal.add.execute.dict_return(
    func=rosenbrock,
    output_param_ids=('rosen') # option if we want to pick certain parts of the returned dict to save to our analysis
) # Execute function (for each branch) with parameters matched to kwargs

anal.add.optimize(
    func=rosenbrock, 
    assign_to='rosen', 
    opt_param_ids=('x', 'y'),
    direction='max', # default is min
    xtol='should xtol and f tol be in options dict or direct args?'
    options={ # set of optional parameters to override defaults
        library: 'scipy',
        method='SLSQP',
        ftol=
        xtol=
        other stuff?
	}
)

# Function wrapper object
anal.add.optimize(
    func=fn.FunctionWrapper(
        rosenbrock, 
        return_type: 'dict',
        assign_to: ,
        positional_inputs: ,
        obj_param
    ), 
    assign_to='rosen', 
    opt_param_ids=('x', 'y'),
    direction='max', # default is min
    xtol='should xtol and f tol be in options dict or direct args?'
    options={ # set of optional parameters to override defaults
        library: 'scipy',
        method='SLSQP',
        ftol=
        xtol=
        other stuff?
	}
)
anal.add.optimize.dict_return(
    func=rosenbrock, 
    obj_param_id='rosen', # pick param to use from the returned dict of func
    opt_param_ids=('x', 'y'),
    direction='max', # default is min
    xtol='should xtol and f tol be in options dict or direct args?'
    options={ # set of optional parameters
        library: 'scipy',
        method='SLSQP',
        ftol=
        xtol=
        other stuff?
	}
)

In [ ]:
### Example #: New Stuff
# Any *analysis step* can be given a conditional function that must return true at runtime or else the *analysis step* will be skipped. One use case for this is when you want to skip an expensive *analysis step* if the parameters aren't looking good.
# As an arbitrary example, assume that we only care about cases where the optimized value of `y` is above 0.5. Also assume `expensive_func` is costly to run and we want to avoid running it when `y<0.5`. 

# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add.fork('a', value_sets=(1, 2))
anal.add.fork.copy()
anal.add.define('x', 0)
anal.add.define('y', 0)

function container object?
fn.FunctionWrapper(func=expensive_func, output_type=single/tuple/dict, )

anal.add.execute(func=expensive_func, output_param_ids='expensive_param')
# multi-output dict vs tuple vs single output
# 

anal.add.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'), save_all_output_keys=true)
# multi-output dict vs single output
# save other dict keys to params

# confusion: we need to make a clear separation between filters that require all branches to catch up in order to continue
# maybe call those filter.comparitive? other ideas: select, rank, sort_and_keep
# actually filter() for not stopping and select() for stopping may be best
# filter.select()??
# select
# should we just not allow custom functions for select because you can always create a param, select from it, then delete it.
anal.add.filter(criteria=lambda y: y>5, drop_unselected=False)
anal.add.filter(criteria=lambda y: y>5, first_absolute=10, drop_unselected=False)
anal.add.filter(criteria=lambda y: y>5, first_portion=0.2, drop_unselected=False)
anal.add.select('y', keep_portion=0.2, mode="greatest", drop_unselected=False) # these are the shortest hand so they must be short
anal.add.select('y', keep_absolute=10, mode="least", drop_unselected=False)
anal.add.select.param('y', keep_portion=0.2, mode="greatest", drop_unselected=False)
anal.add.select.top('y', keep_portion=0.2, drop_unselected=False) # not a fan of .top .bottom for one argument
anal.add.select.bottom('y', keep_absolute=10, drop_unselected=False) # not a fan of .top .bottom for one argument
anal.add.select(func=lambda y: y, keep_absolute=10, mode="least", drop_unselected=False) 

expensive_func = lambda x, y: x+y
anal.add.execute(func=expensive_func, output_param_ids='expensive_param')

results = anal.run()
print(results['df'].drop(columns='datetime'))

done with analysis!
    runtime  a    b         x         y         rosen  expensive_param
0  0.002001  1    0  1.000000  0.000000  4.930381e-32              NaN
1  0.016232  1  100  0.999763  0.999523  5.772481e-08         1.999286
2  0.009002  1  200  0.999939  0.999873  8.146869e-09         1.999811
3  0.001003  2    0  2.000000  0.000000  0.000000e+00              NaN
4  0.010998  2  100  1.999731  3.998866  4.067518e-07         5.998596
5  0.011999  2  200  1.999554  3.998225  2.136755e-07         5.997779


In [ ]:
# Different Output Modes Test

# test rosenbrock with various output formats
def rosenbrock_tuple(x, y, a, b):
    val = rosenbrock(x, y, a, b)
    val2 = 69
    return (val, val2)

def rosenbrock_dict(x, y, a, b):
    val = rosenbrock(x, y, a, b)
    val2 = 69
    return {'rosen_dict': val, 'val2_dict': val2}

# Create new analysis
anal = fn.AnalysisModule()

anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 1))
anal.add(fn.Define('y', 1))

anal.add(fn.Execute(func=rosenbrock, output_param_ids='rosen')) # Direct output mode
anal.add(fn.Execute(func=rosenbrock_tuple, output_param_ids=('rosen_tuple', 'val2_tuple'))) # Positional output mode
anal.add(fn.Execute(func=rosenbrock_dict)) # Keyword output mode
anal.add(fn.Execute(func=fn.dict_handler(rosenbrock_dict, 'rosen'))) # Keyword output mode
anal.add(fn.Execute(func_dict_out=rosenbrock_dict)) # Keyword output mode

results = anal.run()
results['df']


done with analysis!


,runtime,a,b,x,y,rosen,rosen_tuple,val2_tuple,rosen_dict,val2_dict,datetime
0,0.001002,1,100,1,1,0,0,69,0,69,2025-01-09 13:47:02.730670


In [ ]:
# Multi Parameter Forks
# Create new analysis
anal = fn.AnalysisModule()
# Define analysis sequence
anal.add(fn.Define('a', 1)) # Deine parameter
anal.add(fn.Define('b', 100))
# anal.add(fn.Fork(('x', 'y'), value_sets=((0, 1, 2), (0, 1, 2)))) # Fork with new parameter
anal.add(fn.Fork('x', value_sets=((0, 1, 2), (0, 1, 2)))) # Fork with new parameter

results = anal.run()
results['df']


done with analysis!


,runtime,a,b,x,datetime
0,0.0,1,100,"(0, 1, 2)",2025-01-09 13:47:02.757668
1,0.0,1,100,"(0, 1, 2)",2025-01-09 13:47:02.758668


In [ ]:
# Basic Optimization Test
anal = fn.AnalysisModule()

anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', value_sets=(0, 1, 2)))
anal.add(fn.Fork('b', value_sets=(0, 100, 200)))
# anal.add(fn.Fork('b', value_sets=(0, 100, 200), condition=lambda a: a==1))

anal.add(fn.Optimize(func=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y')))

results = anal.run()

print(f"{results['df']:.3f}")

done with analysis!


TypeError: unsupported format string passed to DataFrame.__format__

In [ ]:
# Custom Optimizer Function
import scipy

anal = fn.AnalysisModule()

anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', value_sets=(-1, 0, 1, 2)))

anal.add(fn.Optimize(obj_func=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y'), optimizer=scipy.optimize.minimize, optimizer_args=, optimizer_output=))
anal.add(fn.Optimize(
    optimizer_snippet = lambda paramset:(
        optimizer setup using paramset
    )
))

results = anal.run()

results['df']


## Version 2 ----------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add.define('x', 0)
anal.add.define('y', 0)

anal.add.fork.copy(num_copies=100)

anal.add.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))
anal.add.optimize.dict_return(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'), output_param_ids=('rosen2', 'rosen3'))
anal.add.optimize.custom(optimizer=scipy.optimize.minimize, x0_arg='x0', optimizer_args=dict(
    func=rosenbrock, 
))
# (obj_func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'), optimizer=, , optimizer_output=)

anal.add.execute(output_param_id='expensive_param', func=expensive_func)
anal.add.execute.dict_return(func=expensive_func)
anal.add.execute.tuple_return(output_param_ids=('expensive_param', 'param2'), func=expensive_func)

# Run the analysis sequence
results = anal.run()



TypeError: Optimize.__init__() got an unexpected keyword argument 'optimizer'

In [ ]:


# optimizer mapping tool only

myfunc = lambda a, b, c, d: a+b+c+d
myfunc_kwargs = dict(a=1, b=2, c=3, d=4)

# wrapper function
myfunc_opt, myfunc_x0 = fn.optimize.kwarg_mapper(myfunc, def_kwargs=myfunc_kwargs, opt_vars=('a', 'b'))

# or use a decorator
@optimizer_ready(def_kwargs=myfunc_kwargs, opt_vars=('a', 'b'))
def myfunc_opt(a, b, c, d):
    return myfunc(a, b, c, d)

# or use a dynamic decorator
myfunc_opt = optimizer_ready(def_kwargs=myfunc_kwargs, opt_vars=('a', 'b'))(myfunc)


results = scipy.optimize.minimize(func=myfunc_opt, x0=myfunc_x0)


# what we are up against (custom mapping)
myfunc = lambda a, b, c, d: a+b+c+d

def myfunc_opt(x):
	a = x[0]
	b = x[1]
	c = 3
	d = 4
	return myfunc(a, b, c, d)

myfunc_x0 = [1, 2]

results = scipy.optimize.minimize(func=myfunc_opt, x0=myfunc_x0)



In [ ]:
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Fork('a', value_sets=(1, 2)))
anal.add(fn.Fork('b', value_sets=(0, 100, 200)))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))
anal.add(fn.Optimize(func=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y')))

# Only evaluate 'expensive_func' if the optimized 'x' is above 0.5
expensive_func = lambda x, y: x+y
anal.add(fn.Execute(func=expensive_func, output_param_ids='expensive_param', condition=lambda y: y>0.5))

results = anal.run()
print(results['df'])

done with analysis!
    runtime  a    b         x         y         rosen  \
0  0.004001  1    0  1.000000  0.000000  4.930381e-32   
1  0.009702  1  100  0.999763  0.999523  5.772481e-08   
2  0.017009  1  200  0.999939  0.999873  8.146869e-09   
3  0.000995  2    0  2.000000  0.000000  0.000000e+00   
4  0.016001  2  100  1.999731  3.998866  4.067518e-07   
5  0.020995  2  200  1.999554  3.998225  2.136755e-07   

                    datetime  expensive_param  
0 2025-01-08 00:38:07.088954              NaN  
1 2025-01-08 00:38:07.099666         1.999286  
2 2025-01-08 00:38:07.118677         1.999811  
3 2025-01-08 00:38:07.123193              NaN  
4 2025-01-08 00:38:07.141195         5.998596  
5 2025-01-08 00:38:07.166191         5.997779  


In [ ]:
# Optimization Test with attached objective function
anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', values=(-1, 0, 1, 2)))

anal.add(fn.Define('rosen', attached_func=rosenbrock, function_output_format=('direct', 'dict'), update_other_vars_when_run=True))

# Auto param mapping
anal.add(fn.Optimize('rosen'))

results = anal.run()

results['df']

TypeError: Fork.__init__() got an unexpected keyword argument 'values'

In [ ]:
# Parameter Reference
anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', values=(-1, 0, 1, 2)))

anal.add(fn.Execute(func=rosenbrock, output_param_ids='rosen')) # Direct output mode

anal.add(fn.Define('rosen', attached_func=rosenbrock, function_output_format=('direct', 'dict'), update_other_vars_when_run=True))

anal.add(fn.Fork('opt_method', values=('SLSQP', 'Nelder-Mead')))

anal.add(fn.Function(id='rosen', func=rosenbrock, output_param_ids='rosen_tuple'))


anal.add(fn.Optimize(objective=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y'), method=fn.Reference('opt_method')))



In [ ]:
# Syntax Options
import functioneer as fn

## Version 1: Add steps as Objects ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Fork('a', value_sets=(1, 2))) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add(fn.Fork('b', value_sets=(0, 100, 200))) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Optimize(func=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y')))

# Run the analysis sequence
results = anal.run()
print(results['df'].drop(columns='datetime'))


## Version 2: Add steps through the 'add' namespace ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add.define('x', 0)
anal.add.define('y', 0)

anal.add.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))

# Run the analysis sequence
results = anal.run()


## Version 3: Add steps directly to Analysis ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.define('x', 0)
anal.define('y', 0)

anal.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))

# Run the analysis sequence
results = anal.run()


## Version 4: Declaritive list of Objects ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define the analysis as a configuration
anal = fn.AnalysisModule([
    fn.Fork('a', value_sets=(1, 2)),
    fn.Fork('b', value_sets=(0, 100, 200)),
    fn.Define('x', 0),
    fn.Define('y', 0),
    fn.Execute(method='default', func=rosenbrock),
    fn.Optimize(func=rosenbrock, assign_to='rosen', opt_param_ids=('x', 'y')),
])

# Run the analysis
results = anal.run()


In [ ]:
# pytorch example

# RNN Model
class MadagascarNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        """
        Initializes the MadagascarNN class (Ya Bassic Feed Forward NN) 
        
        Parameters:
        - input_dim (int): Dimensionality of the input features.
        - hidden_dims (list): List of integers, where each integer denotes the number of neurons
                              in a hidden layer. The length of the list determines the number of
                              hidden layers.
        - output_dim (int): Dimensionality of the output.
        """
        super(MadagascarNN, self).__init__()

        # Creating a list of all layers
        self.layers = nn.ModuleList()

        # Input to first hidden layer
        self.layers.append(nn.Linear(input_dim, hidden_dims[0]))
        self.layers.append(nn.ReLU())


In [ ]:
from typing import Literal

def yo(x: Literal['up', 'down'], y: float):
    if x == 'up':
        return y
    return -y



In [ ]:
yo(x='down', y=4)

-4

In [ ]:
### Back Solving
# Back soving is essentilly an optimization of abs(f - value) OR f_zero(f - value)

# Rosenbrock function (known minimum of 0 at: x=1, y=1, a=1, b=100)
def rosenbrock(x, y, a, b):
    return (a-x)**2 + b*(y-x**2)**2

# Create new analysis
anal = fn.AnalysisModule(dict(a=1, b=100))

# Define analysis sequence
anal.add.fork.multi(('x', 'y'), value_sets=((0, 1, 2), (0, 10, 20))) # Fork analysis, create a branch for each value of 'y': 1, 10

# testing out various backsolving apis
anal.add.backsolve(func=rosenbrock, output='rosen', driven_input='a')
anal.add.invert(func=rosenbrock, desired_output='rosen', input='a', method='newton')
anal.add.solve_for(
    func=rosenbrock, 
    target_param='rosen', 
    find_param='a', 
    method='newton',
    tol=1e-6,
    guess=0.5,
    bounds=(0, 10)
    )

anal.add.solve_for(
    func=rosenbrock, 
    target_param='rosen', 
    variable_param='a', 
    method='newton',
    tol=1e-6,
    guess=0.5,
    bounds=(0, 10)
    )

anal.add.solve_for(
    func=rosenbrock, 
    target='rosen', 
    param='a', # or variable
    method='newton',
    tol=1e-6,
    guess=0.5,
    bounds=(0, 10)
    )

anal.add.solve_for(
    func=rosenbrock, 
    target='rosen', 
    solve_for='a', 
    method='newton',
    tol=1e-6,
    guess=0.5,
    bounds=(0, 10)
    )

# Run the analysis sequence
results = anal.run()
print(results['df'].drop(columns='datetime'))